In [ ]:
# Modelling data

In [2]:
# Loading
import pandas as pd
df=pd.read_csv("df_2.csv", sep=",")
df

,recordid,SAPS-I,Creatinine_last_log,SOFA,Age,BUN_last,Na_last,Weight,NIMAP_first,NIMAP_last,...,NISysABP_diff,NIDiasABP_diff,pH_diff,PaO2_diff,PaCO2_diff,In-hospital_death,CCU,CSRU,SICU,Gender
0,-1.508884,-1.342626,-0.356675,-1.264658,-0.604645,-0.847381,-0.663041,NaN,0.727181,0.149738,...,0.673253,0.410868,NaN,NaN,NaN,0,0,0,1,0.0
1,-1.508705,0.300668,0.262364,0.391428,0.654019,-0.216325,-0.893423,-0.226738,-1.706085,-0.573860,...,-1.472152,-0.778341,1.181612,0.943573,-1.443380,0,0,1,0,1.0
2,-1.508526,1.122315,-1.203973,1.101179,-1.176765,-1.090095,-0.202278,-1.047376,1.178184,1.706789,...,-0.399450,-0.155422,0.748908,-1.562412,-0.148834,0,0,0,0,0.0
3,-1.508168,-1.178297,-0.356675,-1.264658,0.196323,-0.750295,-0.432660,0.138935,0.406896,1.421955,...,-0.081612,-1.174743,NaN,NaN,NaN,0,0,0,0,1.0
4,-1.507810,0.464998,0.000000,-1.028074,1.340564,-0.022154,0.028103,NaN,-0.234808,-0.266660,...,0.275956,-0.212051,NaN,NaN,NaN,0,0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5995,1.965126,0.793656,-0.510826,-0.081740,-0.719069,-0.798838,0.258484,-0.056657,NaN,NaN,...,NaN,NaN,-0.008325,1.305013,0.390560,1,0,0,1,1.0
5996,1.865562,-0.192320,-0.356675,-1.028074,1.454988,-0.119239,-2.275711,-0.396818,1.085947,0.785846,...,0.673253,0.127723,NaN,NaN,NaN,0,1,0,0,0.0
5997,1.550394,-0.356650,1.029619,0.628011,0.425171,2.016642,1.180010,4.407952,0.614006,0.435230,...,-0.240531,0.524126,-0.981910,-1.040332,2.440258,0,1,0,0,0.0
5998,0.788082,0.629327,0.000000,1.101179,-0.719069,-0.022154,-1.123804,0.619412,0.708507,-1.231676,...,1.547306,1.543448,-0.332853,-0.630700,-1.119744,1,0,0,0,1.0


In [3]:
# Train test split
from sklearn.model_selection import train_test_split
y=df['In-hospital_death']
x= df-df['In-hospital_death']
X_train, X_test, y_train, y_test=train_test_split(x, y, test_size=0.30, stratify=y, random_state=123)

print(X_train.shape, X_test.shape)
type(y_train)

(4200, 6030) (1800, 6030)


pandas.core.series.Series

In [6]:
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

In [10]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier()
parameters = {'n_neighbors':np.arange(10,100,1000)}

def hyperp_search(classifier, parameters):
    gs = GridSearchCV(classifier, parameters, cv=5, scoring = 'f1', verbose=0, n_jobs=-1)
    gs = gs.fit(X_train, y_train)
    print("f1_train: %f using %s" % (gs.best_score_, gs.best_params_))

    best_model = gs.best_estimator_
    y_pred = best_model.predict(X_test)

    print("f1_test: ", f1_score(y_test, y_pred))
    print(confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred))

In [11]:
hyperp_search(classifier,parameters)

ValueError: 
All the 5 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Alice Donati\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Alice Donati\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neighbors\_classification.py", line 200, in fit
    return self._fit(X, y)
  File "C:\Users\Alice Donati\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neighbors\_base.py", line 407, in _fit
    X, y = self._validate_data(
  File "C:\Users\Alice Donati\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py", li

In [7]:
import numpy as np

from sklearn.ensemble import RandomForestClassifier
classifier=RandomForestClassifier(n_estimators=10, random_state=123)
parameters={'criterion': ['entropy','gini'], 'max_depth': [4,5,10], 'min_samples_leaf': [5, 10],'min_samples_split': [5, 10]}

def hyperp_search(classifier, parameters):
    gs=GridSearchCV(classifier, parameters, cv=3, scoring = 'f1', verbose=0, n_jobs=-1)
    gs=gs.fit(X_train, y_train)
    print("f1_train: %f using %s" %(gs.best_score_, gs.best_params_))

    best_model=gs.best_estimator_
    y_pred=best_model.predict(X_test)

    print("f1_test: ", f1_score(y_test, y_pred))
    print(confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred))

In [8]:
hyperp_search(classifier,parameters)

ValueError: 
All the 72 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
72 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Alice Donati\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Alice Donati\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\ensemble\_forest.py", line 331, in fit
    X, y = self._validate_data(
  File "C:\Users\Alice Donati\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py", line 596, in _validate_data
    X, y = check_X_y(X, y, **check_params)
  File "C:\Users\Alice Donati\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\util

In [9]:
model=RandomForestClassifier(criterion='entropy', max_depth=10 , min_samples_leaf=10, min_samples_split=5)

def roc(model,X_train,y_train,X_test,y_test):
    model.fit(X_train, y_train)
    y_pred=model.predict(X_test)

    y_probs=model.predict_proba(X_test)

    fpr, tpr, thresholds1=metrics.roc_curve(y_test,  y_probs[:,1])

    import matplotlib.pyplot as plt
    plt.plot(fpr, tpr, label='ROC', color='red')
    plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
    plt.xlabel('FP Rate')
    plt.ylabel('TP Rate')
    plt.title('ROC Curve')
    plt.legend()
    plt.show()

    auc=metrics.roc_auc_score(y_test, y_probs[:,1])
    print('AUC: %.2f' % auc)
    return (fpr, tpr)

fpr,tpr=roc(model,X_train,y_train,X_test,y_test)

C:\Users\Alice Donati\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


ValueError: Input X contains NaN.
RandomForestClassifier does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [5]:
#ESEMPIO CON GESTIONE DATI SBILANCAITI
#With SMOTE-Tomek Links method -> combinazione over e undersampling
#esempio sotto

# Define model

import pandas as pd
import numpy as np
from imblearn.pipeline import Pipeline
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_validate
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from imblearn.combine import SMOTETomek
from imblearn.under_sampling import TomekLinks

#Dummy dataset study case
X, Y = make_classification(n_samples=10000, n_features=4, n_redundant=0,
                           n_clusters_per_class=1, weights=[0.99], flip_y=0, random_state=1)

model=RandomForestClassifier(criterion='entropy')
# Define SMOTE-Tomek Links
resample=SMOTETomek(tomek=TomekLinks(sampling_strategy='majority'))
# Define pipeline
pipeline=Pipeline(steps=[('r', resample), ('m', model)])
# Define evaluation procedure (here we use Repeated Stratified K-Fold CV)
cv=RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# Evaluate model
scoring=['accuracy','precision_macro','recall_macro']
scores = cross_validate(pipeline, X, Y, scoring=scoring, cv=cv, n_jobs=-1)
# summarize performance
print('Mean Accuracy: %.4f' % np.mean(scores['test_accuracy']))
print('Mean Precision: %.4f' % np.mean(scores['test_precision_macro']))
print('Mean Recall: %.4f' % np.mean(scores['test_recall_macro']))

Mean Accuracy: 0.9807
Mean Precision: 0.6498
Mean Recall: 0.8418


In [ ]:
# models vari

# SVM

# k-NN

# Tree / Random forest

# Ada boost (?) random forest con diversi modelli

# Neural network composta di pochi dense layer insieme


# all to use with class_weight parameter 

#DecisionTreeClassifier
#LogisticRegression
#RidgeClassifier
#The Keras Python Deep Learning 

In [0]:
# Performances:
# minimum sensitivity and precision
# area under precision-recall curve
# area under reciving operating curve




In [12]:
# Computing sensitivity and precision 
def metrics(): 
    results = []
    
    return results
    

In [13]:
# ROC curve function 
# https://machinelearningmastery.com/roc-curves-and-precision-recall-curves-for-classification-in-python/


def ROC_curve(model, testX, testy)
    # generate a no skill prediction (majority class)
    ns_probs = [0 for _ in range(len(testy))]
    lr_probs = model.predict_proba(testX)
    # keep probabilities for the positive outcome only
    lr_probs = lr_probs[:, 1]
    # calculate scores
    ns_auc = roc_auc_score(testy, ns_probs)
    lr_auc = roc_auc_score(testy, lr_probs)
    # summarize scores
    print('No Skill: ROC AUC=%.3f' % (ns_auc))
    print('Logistic: ROC AUC=%.3f' % (lr_auc))
    # calculate roc curves
    ns_fpr, ns_tpr, _ = roc_curve(testy, ns_probs)
    lr_fpr, lr_tpr, _ = roc_curve(testy, lr_probs)
    # plot the roc curve for the model
    pyplot.plot(ns_fpr, ns_tpr, linestyle='--', label='No Skill')
    pyplot.plot(lr_fpr, lr_tpr, marker='.', label='Logistic')
    # axis labels
    pyplot.xlabel('False Positive Rate')
    pyplot.ylabel('True Positive Rate')
    # show the legend
    pyplot.legend()
    # show the plot
    pyplot.show()

SyntaxError: invalid syntax (637212676.py, line 5)